# Quiz 3: Cleaning “Messy” Data

In this notebook, we explore and clean a wine dataset that contains a variety of data integrity and usability issues. The goal is to perform a thorough exploratory data analysis (EDA), identify and address data quality problems, and prepare the data for use in a machine learning context. The process includes:

- Loading the dataset from a GitHub repository
- Performing EDA with visualizations and summary statistics
- Identifying and addressing missing or invalid values
- Applying appropriate data preparation techniques
- Reviewing the impact of data preparation
- Summarizing key findings and next steps

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer

data_url = "https://github.com/mmonj/CSCI-381-Applied-Data-Science/raw/refs/heads/main/quiz3/data/Day2_Data.csv"
wine_df = pd.read_csv(data_url)


# keep a copy of the original data for later comparison
wine_df_original = wine_df.copy()

wine_df.head()

## Exploratory Data Analysis (EDA)

First, we examine the structure of the dataset, check for missing or unusual values, and generate summary statistics and visualizations for each attribute to help identify any data integrity issues.

In [ ]:
# check the shape and columns
print(f"{wine_df.shape[1]} columns")
wine_df.columns.tolist()

In [ ]:
# check for missing values in each column
wine_df.isna().sum()

In [ ]:
# show summary statistics for all columns
wine_df.describe(include="all")

Below, we visualize the distributions of the numeric features to look for outliers, skewness, and other potential issues.

In [ ]:
# plot histograms for all numeric columns
wine_df.hist(bins=30, figsize=(18, 12))
plt.tight_layout()
plt.show()

In [ ]:
# plot boxplots for all numeric columns to check for outliers
plt.figure(figsize=(18, 12))
for i, col in enumerate(wine_df.select_dtypes(include=[np.number]).columns):
    plt.subplot(4, 4, i+1)
    sns.boxplot(y=wine_df[col])
    plt.title(col)
plt.tight_layout()
plt.show()

From the above, we can see that several columns have missing values and some contain negative or otherwise unusual values (e.g., negative acidity, negative sulfur dioxide, etc.). These will need to be addressed in the data preparation step.

## Data Integrity Issues and Preparation

Based on the EDA, there are several data integrity issues to address:
- Missing values in multiple columns
- Negative values in columns that should be non-negative (e.g., acidity, sulfur dioxide, etc.)
- Possible outliers and skewed distributions

We tackle missing values using imputation (mean for most numeric columns, mode for categorical), and set negative values to NaN before imputation for columns that should only have non-negative values, using the Scikitlearn library.

In [ ]:
nonneg_cols = [
    "FixedAcidity", "VolatileAcidity", "CitricAcid", "ResidualSugar", "Chlorides",
    "FreeSulfurDioxide", "TotalSulfurDioxide", "Density", "pH", "Sulphates", "Alcohol",
    "LabelAppeal", "AcidIndex", "STARS"
]

# set negative values to NaN for these columns
for col in nonneg_cols:
    if col in wine_df.columns:
        wine_df.loc[wine_df[col] < 0, col] = np.nan

# impute missing values with mean for numeric columns, except for STARS, AcidIndex, and LabelAppeal which are discrete/categorical
cat_discrete_cols = ["STARS", "AcidIndex", "LabelAppeal"]
num_cols = [col for col in wine_df.select_dtypes(include=[np.number]).columns if col not in cat_discrete_cols]
imputer = SimpleImputer(strategy="mean")
wine_df[num_cols] = imputer.fit_transform(wine_df[num_cols])

# KNN imputation for STARS to ensure only valid integer values (1-5)
knn_imputer = KNNImputer(n_neighbors=5)
stars_imputed = knn_imputer.fit_transform(wine_df[["STARS"]])
# round to nearest valid integer and clip to valid range
stars_imputed = np.round(stars_imputed).astype(int)
stars_imputed = np.clip(stars_imputed, 1, 5)
wine_df["STARS"] = stars_imputed

# fill missing values in AcidIndex and LabelAppeal with mode (treat as categorical/discrete)
for col in ["AcidIndex", "LabelAppeal"]:
    if col in wine_df.columns:
        wine_df[col].fillna(wine_df[col].mode()[0], inplace=True)

# for any remaining non-numeric columns, fill missing with mode
for col in wine_df.select_dtypes(exclude=[np.number]).columns:
    wine_df[col].fillna(wine_df[col].mode()[0], inplace=True)

# check for missing values again
wine_df.isna().sum()

After imputation, all missing values have been filled. Negative values in columns that should be non-negative have been replaced with the mean of the respective column. For the AcidIndex and LabelAppeal columns, which are discrete or categorical, missing values were filled using the mode (most frequent value). For the STARS column, which is a discrete rating with a large proportion of missing values (~3,000 out of 12,000), KNN imputation was used. 

Mode or median imputation were not ideal for Stars because with so many missing values, these methods would simply fill a large portion of the data with a single value, distorting the actual distribution and reducing variability. KNN imputation makes sure the similarity between samples to estimate missing values based on the patterns in the other features, and the results are rounded and clipped to ensure only valid integer ratings (1-5) are used. This approach avoids introducing nonsensical decimal values and is more robust than median or mode imputation when a large portion of the data is missing.

Below, we rerun EDA on the adjusted columns to compare the results before and after data preparation.

In [ ]:
# replot histograms for the adjusted columns
adj_cols = [col for col in nonneg_cols if col in wine_df.columns]
wine_df[adj_cols].hist(bins=30, figsize=(18, 12))
plt.tight_layout()
plt.show()

In [ ]:
# replot boxplots for the adjusted columns
plt.figure(figsize=(18, 12))
for i, col in enumerate(adj_cols):
    plt.subplot(4, 4, i+1)
    sns.boxplot(y=wine_df[col])
    plt.title(col)
plt.tight_layout()
plt.show()

## Prepped Data Review

The distributions of the adjusted columns now show fewer extreme outliers and no negative values. Imputation has filled in missing data, making the dataset more suitable for machine learning. However, some features remain a bit skewed.

## Conclusions

Through a combination of exploratory data analysis and data preparation, we have identified and addressed several data integrity issues in the wine dataset. Missing and invalid values were handled using imputation, and negative values in non-negative columns were corrected. The resulting dataset is now more suitable for use in machine learning models.